In [3]:
#Importações para fazer a IA escrever poesia
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [4]:
#Fazendo o download do dataset que iremos utilizar como base
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2025-08-14 16:09:34--  https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt
Resolvendo storage.googleapis.com (storage.googleapis.com)... 2800:3f0:4001:801::201b, 2800:3f0:4001:844::201b, 2800:3f0:4001:800::201b, ...
Conectando-se a storage.googleapis.com (storage.googleapis.com)|2800:3f0:4001:801::201b|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 68970 (67K) [text/plain]
Salvando em: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67,35K   301KB/s    em 0,2s    

2025-08-14 16:09:35 (301 KB/s) - ‘/tmp/irish-lyrics-eof.txt’ salvo [68970/68970]



In [6]:
#Tokenização do data set e pegando o tamanho total de palavras
tokenizer = Tokenizer()

data = open('/tmp/irish-lyrics-eof.txt').read()
corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(total_words)

2690


In [7]:
#Separando as sentenças em listas para que ele reconheça o padrão de qual palavra é para vir depois de qual
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

In [9]:
#Adicionando um padding para as listas criadas ficaram exatamente do mesmo tamanho
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [10]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [11]:
#Criação da rede neural e treinamento da mesma
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=100, verbose=1)


2025-08-14 16:17:57.909796: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-14 16:17:57.953468: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-14 16:17:57.957180: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


2025-08-14 16:18:00.243018: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-08-14 16:18:00.354191: I external/local_xla/xla/service/service.cc:168] XLA service 0x7aa99a104ac0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-14 16:18:00.354213: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 6GB Laptop GPU, Compute Capability 8.6
2025-08-14 16:18:00.357972: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755199080.434209   15495 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


377/377 [==============================] - 10s 21ms/step - loss: 6.7518 - accuracy: 0.0651
Epoch 2/100
377/377 [==============================] - 4s 10ms/step - loss: 6.2320 - accuracy: 0.0761
Epoch 3/100
377/377 [==============================] - 3s 8ms/step - loss: 5.9398 - accuracy: 0.0871
Epoch 4/100
377/377 [==============================] - 2s 6ms/step - loss: 5.6293 - accuracy: 0.1013
Epoch 5/100
377/377 [==============================] - 3s 9ms/step - loss: 5.2866 - accuracy: 0.1200
Epoch 6/100
377/377 [==============================] - 3s 9ms/step - loss: 4.9218 - accuracy: 0.1396
Epoch 7/100
377/377 [==============================] - 3s 8ms/step - loss: 4.5538 - accuracy: 0.1642
Epoch 8/100
377/377 [==============================] - 3s 7ms/step - loss: 4.1975 - accuracy: 0.1875
Epoch 9/100
377/377 [==============================] - 3s 7ms/step - loss: 3.8683 - accuracy: 0.2233
Epoch 10/100
377/377 [==============================] - 3s 7ms/step - loss: 3.5426 - accuracy: 0.269

In [12]:
#Demonstração do treinamento, gerando 100 palavras apos essa sentença
seed_text = "I've got a bad feeling about this"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 18ms/step
I've got a bad feeling about this marchin an ra loo ra li youd misfortune without easily easily boyne banbridge sunny railroad marchin whispers bold mccorley mccorley sat father youre at only the ground fearless and the fierce might dare constant be loses there is the wind is hats room is new bride and the young ones turning grey there were more else to be you must to take be from words words it fathers plight or fathers fathers brings thou north rise neatest railroad arm arm rolling captain banbridge brings street middle captain dark railroad holyhead holyhead hats boo wedding says says says to take your
